In [1]:
!pip -q install --upgrade "llama-cpp-python[server]==0.2.84" google-adk litellm huggingface_hub python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 13.7 MB/s eta 0:00:00


In [2]:
import os
from huggingface_hub import hf_hub_download

# DeepSeek-Coder 6.7B Instruct (quantized GGUF) – good balance of quality/speed
REPO_ID = "TheBloke/DeepSeek-Coder-6.7B-Instruct-GGUF"
FILENAME = "deepseek-coder-6.7b-instruct.Q4_K_M.gguf"

model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
model_path


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


deepseek-coder-6.7b-instruct.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf'

In [3]:
# run this once so the next bash cell can see the path
import os
os.environ["MODEL_PATH"] = model_path  # model_path comes from Step 2
print("Using model at:", os.environ["MODEL_PATH"])


Using model at: /root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf


In [4]:
%%bash
# 3.A1 — start server in background and log to /content/server.log
pkill -f "llama_cpp.server.*--port 8000" || true

nohup python -m llama_cpp.server \
  --model "$MODEL_PATH" \
  --host 127.0.0.1 --port 8000 \
  --model_alias deepseek-local \
  --chat_format chatml \
  --n_ctx 4096 \
  > /content/server.log 2>&1 &

sleep 3
tail -n 50 /content/server.log


In [5]:
# 3.A2 — wait for server to be ready
import time, requests, subprocess

base = "http://127.0.0.1:8000"
for i in range(120):  # up to ~6 minutes on CPU
    try:
        r = requests.get(f"{base}/v1/models", timeout=2)
        if r.ok:
            print("Server is ready ✅")
            print(r.json())
            break
    except Exception:
        pass
    if i % 5 == 0:
        print(f"...waiting ({i*3}s)")
    time.sleep(3)
else:
    print("Server not ready; last 100 log lines:")
    subprocess.run(["tail","-n","100","/content/server.log"])


...waiting (0s)
...waiting (15s)
Server is ready ✅
{'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [6]:
# 3.A3 — set env for ADK/LiteLLM
import os
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"]  = "sk-local-anything"  # dummy
print("Env set. Proceed to the ADK agent + FunctionTool cell.")


Env set. Proceed to the ADK agent + FunctionTool cell.


In [8]:
import requests
r = requests.post(
    "http://127.0.0.1:8000/v1/chat/completions",
    headers={"Authorization": "Bearer sk-local-anything"},
    json={
        "model": "deepseek-local",
        "messages": [{"role":"user","content":"Say hello in 1 short sentence."}],
        "max_tokens": 32,
    },
    timeout=30,
)
print(r.status_code, r.json()["choices"][0]["message"]["content"])


200 Hello, how can I assist you today?


In [9]:
import os
# LiteLLM timeouts (seconds). Keeps requests from hanging indefinitely.
os.environ["LITELLM_TIMEOUT"] = "120"
os.environ["LITELLM_REQUEST_TIMEOUT"] = "120"   # some versions read this name
os.environ["LITELLM_MAX_RETRIES"] = "0"         # fail fast if slow


In [16]:
from typing import Dict, Any

def estimate_kitchen_budget(square_feet: int, finish_level: str) -> Dict[str, Any]:
    finish_level = (finish_level or "standard").lower().strip()
    cost_per_sf = {"basic": 150, "standard": 300, "premium": 600}.get(finish_level, 300)
    base = 4000
    estimate = base + square_feet * cost_per_sf
    line_items = [
        {"item": "Design & Permits", "cost": int(0.07 * estimate)},
        {"item": "Cabinetry & Millwork", "cost": int(0.30 * estimate)},
        {"item": "Appliances", "cost": int(0.18 * estimate)},
        {"item": "Counters & Backsplash", "cost": int(0.15 * estimate)},
        {"item": "Plumbing & Electrical", "cost": int(0.12 * estimate)},
        {"item": "Flooring/Paint/Misc.", "cost": int(0.08 * estimate)},
        {"item": "Contingency", "cost": int(0.10 * estimate)},
    ]
    return {
        "status": "success",
        "estimate_usd": int(estimate),
        "finish_level": finish_level,
        "inputs": {"square_feet": square_feet, "finish_level": finish_level},
        "line_items": line_items,
        "notes": "Heuristic only—validate with contractor bids.",
    }

def materials_recommender(finish_level: str) -> Dict[str, Any]:
    finish_level = (finish_level or "standard").lower().strip()
    if finish_level == "basic":
        counters = "Laminate or butcher-block"
        cabinets = "Stock shaker (MDF), pre-finished"
        flooring = "Vinyl plank"
        hardware = "Brushed nickel, value line"
        note = "Targets affordability with durable but budget materials."
    elif finish_level == "premium":
        counters = "Quartzite or Dekton"
        cabinets = "Custom hardwood, soft-close, full-overlay"
        flooring = "Engineered oak/walnut, water-resistant finish"
        hardware = "Solid brass, premium finish"
        note = "High-end aesthetics and longevity; lead times may impact schedule."
    else:
        counters = "Quartz (durable, low-maintenance)"
        cabinets = "Semi-custom, plywood boxes, soft-close"
        flooring = "Porcelain tile or mid-tier engineered wood"
        hardware = "Matte black or satin brass"
        note = "Balanced cost/quality; widely available SKUs."
    return {
        "finish_level": finish_level,
        "materials": [
            {"category": "Countertops", "recommendation": counters},
            {"category": "Cabinetry", "recommendation": cabinets},
            {"category": "Flooring", "recommendation": flooring},
            {"category": "Hardware", "recommendation": hardware},
        ],
        "note": note,
    }

def timeline_generator(square_feet: int, finish_level: str) -> Dict[str, Any]:
    finish_factor = {"basic": 0.9, "standard": 1.0, "premium": 1.2}.get((finish_level or "standard").lower(), 1.0)
    base_weeks = 6.0
    size_factor = max(0.75, min(1.35, square_feet / 180.0))
    total_weeks = base_weeks * size_factor * finish_factor
    design = max(0.8, round(0.20 * total_weeks, 1))
    procurement = max(1.0, round(0.25 * total_weeks, 1))
    build = max(2.5, round(0.55 * total_weeks, 1))
    return {
        "total_weeks": float(round(design + procurement + build, 1)),
        "phases": [
            {"name": "Design & Permits", "duration_weeks": design, "milestones": ["Layout confirm", "Selections", "Permit submit"]},
            {"name": "Procurement", "duration_weeks": procurement, "milestones": ["Cabinets order", "Counter template window", "Appliances"]},
            {"name": "Build & Install", "duration_weeks": build, "milestones": ["Demo", "Rough-ins", "Cabinet set", "Counters", "Punch list"]},
        ],
        "assumptions": [
            "No structural changes and no full rewire.",
            "Lead times typical; expedite may change costs.",
        ],
    }

# quick smoke tests
print("Budget sample:", estimate_kitchen_budget(180, "standard")["estimate_usd"])
print("Materials sample:", materials_recommender("standard")["materials"][0])
print("Timeline sample (weeks):", timeline_generator(180, "standard")["total_weeks"])


Budget sample: 58000
Materials sample: {'category': 'Countertops', 'recommendation': 'Quartz (durable, low-maintenance)'}
Timeline sample (weeks): 6.0


In [18]:
# === ADK agent + FunctionTool + Markdown fallback (single cell) ===
import asyncio
from dataclasses import dataclass
from typing import Dict, Any, List

# ADK bits
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types

# Route ADK → your local OpenAI-compatible server via LiteLLM
from google.adk.models.lite_llm import LiteLlm

# ---------- Simple Function Tool ----------
@dataclass
class BudgetInput:
    square_feet: int
    finish_level: str  # "basic" | "standard" | "premium"

def estimate_kitchen_budget(square_feet: int, finish_level: str) -> Dict[str, Any]:
    finish_level = (finish_level or "standard").lower().strip()
    cost_per_sf = {"basic": 150, "standard": 300, "premium": 600}.get(finish_level, 300)
    base = 4000
    estimate = base + square_feet * cost_per_sf
    line_items = [
        {"item": "Design & Permits", "cost": int(0.07 * estimate)},
        {"item": "Cabinetry & Millwork", "cost": int(0.30 * estimate)},
        {"item": "Appliances", "cost": int(0.18 * estimate)},
        {"item": "Counters & Backsplash", "cost": int(0.15 * estimate)},
        {"item": "Plumbing & Electrical", "cost": int(0.12 * estimate)},
        {"item": "Flooring/Paint/Misc.", "cost": int(0.08 * estimate)},
        {"item": "Contingency", "cost": int(0.10 * estimate)},
    ]
    return {
        "status": "success",
        "estimate_usd": int(estimate),
        "finish_level": finish_level,
        "inputs": {"square_feet": square_feet, "finish_level": finish_level},
        "line_items": line_items,
        "notes": "Heuristic only—validate with contractor bids.",
    }

budget_tool = FunctionTool(func=estimate_kitchen_budget)

# ---------- LLM + Agent ----------
local_model = LiteLlm(model="openai/deepseek-local")

kitchen_agent = LlmAgent(
    model=local_model,
    name="kitchen_renovation_agent",
    description="Creates a concise kitchen renovation proposal and uses a budget estimator tool.",
    instruction=(
        "You are a renovation consultant. Produce a 1-page proposal with sections:\n"
        "1) Scope overview; 2) Design concept; 3) Materials (bullets);\n"
        "4) Timeline (phases); 5) Budget summary; 6) Disclaimers.\n"
        "Always call the tool `estimate_kitchen_budget(square_feet, finish_level)` to compute the budget.\n"
        "If the tool returns 'status: success', include total and line items.\n"
        "Write clear, structured Markdown. Be practical and specific."
    ),
    tools=[budget_tool],
)

APP_NAME = "adk_kitchen_agent_demo"
USER_ID = "student"
SESSION_ID = "session-001"

# ---------- Fallback synthesizer (guaranteed Markdown) ----------
def synthesize_markdown(square_feet: int, finish_level: str, notes: str) -> str:
    b = estimate_kitchen_budget(square_feet, finish_level)
    line_items_md = "\n".join(f"- {li['item']}: ${li['cost']}" for li in b["line_items"])
    scope = f"Renovate existing kitchen; retain layout if feasible; {notes or 'island seating for 3; quartz counters.'}"
    return (
f"# Kitchen Renovation Proposal — {square_feet} sq ft ({b['finish_level']})\n\n"
f"## 1) Scope Overview\n- {scope}\n\n"
"## 2) Design Concept\n- Clean, functional, durable finishes; good task lighting; cohesive hardware.\n\n"
"## 3) Materials\n"
"- Quartz counters; semi-custom cabinets (soft-close); mid-tier flooring; matte black or satin brass hardware.\n\n"
"## 4) Timeline\n"
"- Design & Permits: ~1–2 weeks\n"
"- Procurement: ~2–3 weeks\n"
"- Build & Install: ~3–4 weeks\n\n"
f"## 5) Budget Summary (USD)\n- Estimated total: **${b['estimate_usd']}**\n{line_items_md}\n\n"
"## 6) Assumptions & Disclaimers\n"
"- No structural changes or full rewire assumed.\n"
"- Quotes are heuristic; validate with contractor bids and current lead times.\n"
    )

# ---------- Helper: detect refusal/weak responses ----------
REFUSAL_MARKERS = [
    "i'm sorry", "i am sorry", "i cannot", "i can’t", "i can't",
    "do not have the capability", "as an ai model", "unable to", "cannot generate",
    "do not have access to real-time", "don't have access to real-time"
]

def looks_like_refusal(text: str) -> bool:
    t = (text or "").lower()
    if not t.strip():
        return True
    if any(m in t for m in REFUSAL_MARKERS):
        return True
    # Too short or missing key sections
    needed = ["scope", "design concept", "materials", "timeline", "budget", "assumptions"]
    miss = sum(1 for k in needed if k not in t)
    return miss >= 3  # missing most sections

# ---------- Runner (single turn) with fallback ----------
async def call_agent_with_fallback(square_feet: int, finish_level: str, notes: str) -> str:
    session_service = InMemorySessionService()
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=kitchen_agent, app_name=APP_NAME, session_service=session_service)

    # User prompt the agent sees (it should call the tool itself)
    user_prompt = (
        f"Create a kitchen renovation proposal.\n"
        f"- square_feet: {square_feet}\n- finish_level: {finish_level}\n- notes: {notes}\n"
        f"- target_budget: under $45k if feasible\n"
        "Follow the required sections and ALWAYS call the budget tool."
    )
    content = types.Content(role="user", parts=[types.Part(text=user_prompt)])

    final_text = None

    def extract_text(content_obj):
        try:
            if content_obj and getattr(content_obj, "parts", None):
                for p in content_obj.parts:
                    t = getattr(p, "text", None)
                    if t:
                        return t
        except Exception:
            pass
        return None

    for event in runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        # Optional: debug print tool calls
        tool_name = getattr(event, "tool_name", None)
        parameters = getattr(event, "parameters", None)
        if tool_name and parameters is not None:
            print("→ Tool request:", tool_name, parameters)

        tool_resp = getattr(event, "tool_response", None)
        if tool_resp is not None:
            print("← Tool response (obj):", str(tool_resp)[:180], "...")
        else:
            txt = extract_text(getattr(event, "content", None))
            if txt and ("estimate_usd" in txt or "line_items" in txt or "status" in txt):
                print("← Tool response (preview):", txt[:180], "...")

        # Capture assistant text as it streams
        txt = extract_text(getattr(event, "content", None))
        if txt:
            final_text = txt

    # Fallback if refusal/weak
    if looks_like_refusal(final_text):
        print("\n[Fallback] Synthesizing Markdown locally…")
        final_text = synthesize_markdown(square_feet, finish_level, notes)

    print("\n===== Final Proposal =====\n")
    print(final_text)
    return final_text

# ----- Run once (Colab supports top-level await) -----
final_md = await call_agent_with_fallback(
    square_feet=180,
    finish_level="standard",
    notes="Island seating for 3; durable quartz; keep existing layout if possible."
)



[Fallback] Synthesizing Markdown locally…

===== Final Proposal =====

# Kitchen Renovation Proposal — 180 sq ft (standard)

## 1) Scope Overview
- Renovate existing kitchen; retain layout if feasible; Island seating for 3; durable quartz; keep existing layout if possible.

## 2) Design Concept
- Clean, functional, durable finishes; good task lighting; cohesive hardware.

## 3) Materials
- Quartz counters; semi-custom cabinets (soft-close); mid-tier flooring; matte black or satin brass hardware.

## 4) Timeline
- Design & Permits: ~1–2 weeks
- Procurement: ~2–3 weeks
- Build & Install: ~3–4 weeks

## 5) Budget Summary (USD)
- Estimated total: **$58000**
- Design & Permits: $4060
- Cabinetry & Millwork: $17400
- Appliances: $10440
- Counters & Backsplash: $8700
- Plumbing & Electrical: $6960
- Flooring/Paint/Misc.: $4640
- Contingency: $5800

## 6) Assumptions & Disclaimers
- No structural changes or full rewire assumed.
- Quotes are heuristic; validate with contractor bids and current